In [2]:
import pandas as pd

In [3]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "schools_complete.csv"
student_data_to_load = "students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.tail(10)

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
39160,39160,Katie Weaver,F,11th,Thomas High School,89,86,14,Charter,1635,1043130
39161,39161,April Reyes,F,10th,Thomas High School,70,84,14,Charter,1635,1043130
39162,39162,Derek Weeks,M,12th,Thomas High School,94,77,14,Charter,1635,1043130
39163,39163,John Reese,M,11th,Thomas High School,90,75,14,Charter,1635,1043130
39164,39164,Joseph Anthony,M,9th,Thomas High School,97,76,14,Charter,1635,1043130
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,1635,1043130
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,1635,1043130
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,1635,1043130
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,1635,1043130
39169,39169,Carolyn Jackson,F,11th,Thomas High School,95,75,14,Charter,1635,1043130


In [4]:
#District Summary
#create data frame
schools_df = pd.DataFrame(school_data_complete)
#school count
school_count = len(schools_df["school_name"].unique())
#student_count
stu_count = len(schools_df["student_name"])
#need to get sum of budget but NOT duplicate budgets within schools. Dropping duplicates and creating a group
budgey = schools_df.drop_duplicates().groupby("budget").sum()
#creating a dataframe to get budget sum
budgey_df = pd.DataFrame(budgey)
#need to move the budget field out of the index column
budgey_df.reset_index(inplace=True)
#converting budget from string to int
budgey_df["budget"] = pd.to_numeric(budgey_df["budget"]) 
#getting the budget sum
budgey_df["budget"].sum()
#average math score
avemath = schools_df["math_score"].mean()
#average reading score
averead = schools_df["reading_score"].mean()
#begin the process for percent passing math by putting into bins because why not?
mathbin = [0, 70, 100]
mathlabe = ["0", "1"]
mathperc = pd.cut(schools_df["math_score"], mathbin, labels=mathlabe)
mathperc_df = pd.DataFrame(mathperc)
#convert math percentage to int and get the sum
mathperc_df["math_score"] = pd.to_numeric(mathperc_df["math_score"]) 
mathperc_df["math_score"].sum()
#percentage passing math
mathpass = (mathperc_df["math_score"].sum()/stu_count) * 100
mathpass
#begin process for percent passing reading by putting into bins
readbin = [0, 69, 100]
readlabe = ["0", "1"]
readperc = pd.cut(schools_df["reading_score"], readbin, labels=readlabe)
readperc_df = pd.DataFrame(readperc)
#convert reading percentage to int and get the sum
readperc_df["reading_score"] = pd.to_numeric(readperc_df["reading_score"]) 
readperc_df["reading_score"].sum()
#percentage passing math
readpass = (readperc_df["reading_score"].sum()/stu_count) * 100
#calculate overall passing
overallpass = (mathpass + readpass)/2
overallpass
#create the final dataframe
district_df = pd.DataFrame({
     "Total Schools": [school_count],
     "Total Students": [stu_count],
     "Total Budget": [budgey_df["budget"].sum()],
     "Average Math Score": [avemath],
     "Average Reading Score": [averead],
     "% Passing Math": [mathpass],
     "% Passing Reading": [readpass],
     "% Overall Passing Rate": [overallpass]   
    })
district_df


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,24649428,78.985371,81.87784,72.392137,85.805463,79.0988


In [5]:
#School Summary
#start with the original data, build a new frame
school = school_data_complete[["school_name", "type", "Student ID", "budget", "math_score", "reading_score"]]
#do the calculated columns
school_calcs = school.groupby(['school_name','type','budget']).agg({'Student ID':'count','math_score': 'mean', 
                                                              'reading_score': 'mean'}).reset_index()
school_calcs

,school_name,type,budget,Student ID,math_score,reading_score
0,Bailey High School,District,3124928,4976,77.048432,81.033963
1,Cabrera High School,Charter,1081356,1858,83.061895,83.975780
2,Figueroa High School,District,1884411,2949,76.711767,81.158020
3,Ford High School,District,1763916,2739,77.102592,80.746258
4,Griffin High School,Charter,917500,1468,83.351499,83.816757
5,Hernandez High School,District,3022020,4635,77.289752,80.934412
6,Holden High School,Charter,248087,427,83.803279,83.814988
7,Huang High School,District,1910635,2917,76.629414,81.182722
8,Johnson High School,District,3094650,4761,77.072464,80.966394
9,Pena High School,Charter,585858,962,83.839917,84.044699


In [6]:
mathpass = school_data_complete[["school_name", "math_score"]]
students = mathpass["school_name"]
passgrades = mathpass[(mathpass["math_score"] >= 70)]
#group by school
finalmath = passgrades.groupby(["school_name"]).count().reset_index()
#create a frame for the passing reading
finalmath

,school_name,math_score
0,Bailey High School,3318
1,Cabrera High School,1749
2,Figueroa High School,1946
3,Ford High School,1871
4,Griffin High School,1371
5,Hernandez High School,3094
6,Holden High School,395
7,Huang High School,1916
8,Johnson High School,3145
9,Pena High School,910


In [7]:
#School Summary
#start with the original data, build a new frame
school = school_data_complete[["school_name", "type", "Student ID", "budget", "math_score", "reading_score"]]
#do the calculated columns
school_calcs = school.groupby(['school_name','type','budget']).agg({'Student ID':'count','math_score': 'mean', 
                                                              'reading_score': 'mean'}).reset_index()
#create a frame for the passing math
mathpass = school_data_complete[["school_name", "math_score"]]
students = mathpass["school_name"]
passgrades = mathpass[(mathpass["math_score"] >= 70)]
#group by school
finalmath = passgrades.groupby(["school_name"]).count().reset_index()
#create a frame for the passing reading
readpass = school_data_complete[["school_name", "reading_score"]]
passgradesr = readpass[(readpass["reading_score"] >= 70)]
#group by school
finalread = passgradesr.groupby(["school_name"]).count().reset_index()
#merge the two
mergetables = pd.merge(finalmath, finalread, how="left", on=["school_name", "school_name"])
mergetables
#rename some of the columns to avoid confusion in columns when calculating
school_calcs = school_calcs.rename(columns = {"school_name":"school_name", "type":"type", "budget":"budget", 
                "Student ID":"Tot Students", "math_score":"average_math", "reading_score":"average_reading"})                                         

#gather everything together and calculate                          
school_calcs
schools_summ = pd.merge(school_calcs, mergetables, how="left", on=["school_name", "school_name"])
schools_summ
schools_summ['% Passing Math'] = round(schools_summ["math_score"]/schools_summ["Tot Students"]*100,2)
schools_summ['% Passing Reading'] = round(schools_summ['reading_score']/schools_summ['Tot Students']*100,2)
schools_summ['% Overall Passing'] = round((schools_summ['math_score'] + schools_summ['reading_score']) / 2,2)
schools_summ['Per Student Budget'] = schools_summ['budget']/schools_summ['Tot Students']
#final table
final = schools_summ[['school_name', 'type', 'Tot Students', 'budget', 
                          'Per Student Budget', 'math_score', 'reading_score', 
                          '% Passing Math', '% Passing Reading', '% Overall Passing']]

final

,school_name,type,Tot Students,budget,Per Student Budget,math_score,reading_score,% Passing Math,% Passing Reading,% Overall Passing
0,Bailey High School,District,4976,3124928,628.0,3318,4077,66.68,81.93,3697.5
1,Cabrera High School,Charter,1858,1081356,582.0,1749,1803,94.13,97.04,1776.0
2,Figueroa High School,District,2949,1884411,639.0,1946,2381,65.99,80.74,2163.5
3,Ford High School,District,2739,1763916,644.0,1871,2172,68.31,79.30,2021.5
4,Griffin High School,Charter,1468,917500,625.0,1371,1426,93.39,97.14,1398.5
5,Hernandez High School,District,4635,3022020,652.0,3094,3748,66.75,80.86,3421.0
6,Holden High School,Charter,427,248087,581.0,395,411,92.51,96.25,403.0
7,Huang High School,District,2917,1910635,655.0,1916,2372,65.68,81.32,2144.0
8,Johnson High School,District,4761,3094650,650.0,3145,3867,66.06,81.22,3506.0
9,Pena High School,Charter,962,585858,609.0,910,923,94.59,95.95,916.5


In [8]:
#top five performing schools
final.nlargest(5, '% Overall Passing')

,school_name,type,Tot Students,budget,Per Student Budget,math_score,reading_score,% Passing Math,% Passing Reading,% Overall Passing
0,Bailey High School,District,4976,3124928,628.0,3318,4077,66.68,81.93,3697.5
8,Johnson High School,District,4761,3094650,650.0,3145,3867,66.06,81.22,3506.0
5,Hernandez High School,District,4635,3022020,652.0,3094,3748,66.75,80.86,3421.0
10,Rodriguez High School,District,3999,2547363,637.0,2654,3208,66.37,80.22,2931.0
13,Wilson High School,Charter,2283,1319574,578.0,2143,2204,93.87,96.54,2173.5


In [9]:
#bottom five performing schools
final.nsmallest(5, '% Overall Passing')

,school_name,type,Tot Students,budget,Per Student Budget,math_score,reading_score,% Passing Math,% Passing Reading,% Overall Passing
6,Holden High School,Charter,427,248087,581.0,395,411,92.51,96.25,403.0
9,Pena High School,Charter,962,585858,609.0,910,923,94.59,95.95,916.5
4,Griffin High School,Charter,1468,917500,625.0,1371,1426,93.39,97.14,1398.5
12,Thomas High School,Charter,1635,1043130,638.0,1525,1591,93.27,97.31,1558.0
11,Shelton High School,Charter,1761,1056600,600.0,1653,1688,93.87,95.85,1670.5


In [10]:
#Math Scores by Grade
#prepare for math score by grade by setting up new frame 
grades_math_set = school_data_complete[["school_name", "math_score", "grade"]]
#calculate
math_grades = grades_math_set.groupby(['school_name', 'grade']).agg({'math_score': 'mean'}).reset_index()
#create a pivot table 
mathpivot = math_grades.pivot_table('math_score', ['school_name'], 'grade').reset_index()
#create new frame, final result
mathtable = mathpivot[['school_name','9th', '10th', '11th', '12th']]
mathtable


grade,school_name,9th,10th,11th,12th
0,Bailey High School,77.083676,76.996772,77.515588,76.492218
1,Cabrera High School,83.094697,83.154506,82.765560,83.277487
2,Figueroa High School,76.403037,76.539974,76.884344,77.151369
3,Ford High School,77.361345,77.672316,76.918058,76.179963
4,Griffin High School,82.044010,84.229064,83.842105,83.356164
5,Hernandez High School,77.438495,77.337408,77.136029,77.186567
6,Holden High School,83.787402,83.429825,85.000000,82.855422
7,Huang High School,77.027251,75.908735,76.446602,77.225641
8,Johnson High School,77.187857,76.691117,77.491653,76.863248
9,Pena High School,83.625455,83.372000,84.328125,84.121547


In [11]:
#Reading Scores by Grade
grades_read_set = school_data_complete[["school_name", "reading_score", "grade"]]
# calculate 
read_grades = grades_read_set.groupby(['school_name', 'grade']).agg({'reading_score': 'mean'}).reset_index()
#create a pivot table 
readpivot = read_grades.pivot_table('reading_score', ['school_name'], 'grade').reset_index()
#create new frame, final result 
readtable = readpivot[['school_name','9th', '10th', '11th', '12th']]
readtable

grade,school_name,9th,10th,11th,12th
0,Bailey High School,81.303155,80.907183,80.945643,80.912451
1,Cabrera High School,83.676136,84.253219,83.788382,84.287958
2,Figueroa High School,81.198598,81.408912,80.640339,81.384863
3,Ford High School,80.632653,81.262712,80.403642,80.662338
4,Griffin High School,83.369193,83.706897,84.288089,84.013699
5,Hernandez High School,80.866860,80.660147,81.396140,80.857143
6,Holden High School,83.677165,83.324561,83.815534,84.698795
7,Huang High School,81.290284,81.512386,81.417476,80.305983
8,Johnson High School,81.260714,80.773431,80.616027,81.227564
9,Pena High School,83.807273,83.612000,84.335938,84.591160


In [12]:
#Scores by School Spending
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]
#dataframe from last final table
spending = final[['Per Student Budget', "math_score", "reading_score", 
                                  '% Passing Math', '% Passing Reading', '% Overall Passing']]
#cut into bins
spending['Spending Ranges'] = pd.cut(spending["Per Student Budget"], spending_bins, labels=group_names)
#group and show average
scores = spending.groupby(['Spending Ranges']).mean().reset_index()
#create new dataframe
scores_by_school = scores[['Spending Ranges', "math_score", "reading_score", 
                          '% Passing Math', '% Passing Reading', '% Overall Passing']]
#group and average again
scores_by_school.groupby(['Spending Ranges']).mean()

C:\Users\Kimberly\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,math_score,reading_score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges,,,,,
<$585,1491.750000,1539.250000,93.460000,96.610000,1515.500000
$585-615,1281.500000,1305.500000,94.230000,95.900000,1293.500000
$615-645,2114.166667,2475.833333,75.668333,86.106667,2295.000000
$645-675,2718.333333,3329.000000,66.163333,81.133333,3023.666667


In [13]:
# Scores by School Size
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
#dataframe from last final table
size = final[['Tot Students', 'math_score', 'reading_score', 
                                  '% Passing Math', '% Passing Reading', '% Overall Passing']]
#cut into bins
size['School Size'] = pd.cut(size["Tot Students"], size_bins, labels=group_names)
#group and show average
sizes = size.groupby(['School Size']).mean().reset_index()
#create new dataframe
score_by_size = sizes[['School Size', 'math_score', 'reading_score', 
                          '% Passing Math', '% Passing Reading', '% Overall Passing']]
#group and give average again
score_by_size.groupby(['School Size']).mean()

C:\Users\Kimberly\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,math_score,reading_score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),652.500,667.000,93.55000,96.10000,659.75
Medium (1000-2000),1595.600,1649.400,93.59800,96.79000,1622.50
Large (2000-5000),2510.875,3003.625,69.96375,82.76625,2757.25


In [14]:
#Scores by School Type
#dataframe from last final table
stype = final[['type', 'math_score', 'reading_score', 
                                  '% Passing Math', '% Passing Reading', '% Overall Passing']]

#group and show average
types = stype.groupby(['type']).mean()
types

,math_score,reading_score,% Passing Math,% Passing Reading,% Overall Passing
type,,,,,
Charter,1428.250000,1473.125000,93.620000,96.586250,1450.687500
District,2563.428571,3117.857143,66.548571,80.798571,2840.642857
